# Problem 2 - KNN hyperparameter tuning using cross validation 20 points
For this problem you should read the article at: https://www.analyticsvidhya.com/blog/2021/01/a-quick- introduction-to-k-nearest-neighbor-knn-classification-using-python/ to review how to work with K-Nearest Neighbor (KNN) in sklearn. We will use the same Social Network ads dataset that is used in this post. You will work with a 80-20 train-test split.
You will use KNN algorithm to predict whether an individual will buy a product or not. As discussed in the class, there are two hyperparameters: the number of neighbors $(K)$ and the distance metric. For distance between two n-dimensional points $\bar{x_1} = {x_{1,1}, x_{1,2}, . . . , x_{1,n}}$ and $\bar{x_2} = {x_{2,1}, x_{2,2}, . . . , x_{2,n}}$ we consider Minkowski distance given by:

$( \sum_{i=1}^{n} |x_{1,i}-x_{2,i}|^p)^{\frac{1}{p}}$

where p is a parameter. For p = 2, this distance is same as Euclidean distance and for p = 1 it is called
Manhattan distance.


### 1-
With K = 4 and p = 2 train a KNN classifier and evaluate its misclassification error, Accuracy,
Precision, Recall, F-1 score on the test set.

### 2- 
You will use 5-fold cross-validation to identify the best value of K. First fix p = 1 and for K ∈ [1,2,...,15] calculate the misclassification error and plot it as a function of K for different values of K.

### 3-
Next fix p = 2 and again using 5-fold cross-validation for K ∈ [1, 2, . . . , 15] calculate the misclassification error and plot it as a function of K. This should be plotted in the same graph as for p = 1 in part 2 of this problem. 

### 4- 
What is the best value of K with Eucledian distance? Is this value the same with Manhattan distance? What combination of p and K gives the best classifier (one with the minimum misclassification error).